<a href="https://colab.research.google.com/github/YasashiiDia/InteractiveBorda/blob/main/books_compiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown Imports

import math
import numpy as np
import pandas as pd
import requests
import json
import seaborn as sns; sns.set()
import ipywidgets as widgets
from IPython.display import HTML
import gspread
from gspread_dataframe import set_with_dataframe
import matplotlib.pyplot as plt
from collections import Counter

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

from google.colab import drive
drive.mount('/content/drive')

!pip install unidecode
import unidecode

In [182]:
from urllib.request import urlopen

with open("/content/drive/MyDrive/google_books_api_key.txt", "r") as f:
  key = f.read()

def query_books_api(author, title):
  
  api = f"https://www.googleapis.com/books/v1/volumes?q={title}+inauthor:{author}&key={key}"
  api = api.replace(" ", "+")

  # send a request and get a JSON response
  resp = urlopen(api)
  # parse JSON into Python as a dictionary
  book_data = json.load(resp)

  # create additional variables for easy querying
  volume_info = book_data["items"][0]["volumeInfo"]
  author = volume_info["authors"]
  # practice with conditional expressions!
  prettify_author = author if len(author) > 1 else author[0]

  # find ISBN13, if not available, use first ID type
  IDs = volume_info['industryIdentifiers']#[0]['identifier']
  for ID in IDs:
    if ID["type"] == "ISBN_13":
      ID = ID["identifier"]
      break
  else:
    ID = volume_info['industryIdentifiers'][0]['identifier']

  # print(f"\nTitle: {volume_info['title']}")
  # print(f"Author: {prettify_author}")
  # print(f"Page Count: {volume_info['pageCount']}")
  # print(f"Publication Date: {volume_info['publishedDate']}")
  # print(f"ID: {ID}")
  return ID, volume_info['title'], prettify_author

In [164]:
SHEETNAME = "books_sheet"
MAX_VOTES = 50
votesheet = gc.open(SHEETNAME).worksheet('Votes')
idvotesheet = gc.open(SHEETNAME).worksheet('ID_Votes')

In [ ]:
votes = np.array(votesheet.get_all_values())
votes_df = pd.DataFrame(columns=votes[0], index=range(MAX_VOTES))
votes_df.iloc[:votes[1:,:].shape[0], :votes[1:,:].shape[1]] = votes[1:,:]
votes_df.head()

In [ ]:
idvotes = np.array(idvotesheet.get_all_values())
idvotes_df = pd.DataFrame(columns=idvotes[0], index=range(MAX_VOTES))
idvotes_df.iloc[:idvotes[1:,:].shape[0], :idvotes[1:,:].shape[1]] = idvotes[1:,:]
idvotes_df.head()

In [178]:
voters_to_update = []
update_all_votes = False # if false, update only newly appended votes if some converted votes already exist
force_update_all = False

for voter in votes_df:

  if voter not in idvotes_df.columns or voter in voters_to_update or (len(votes_df[voter].dropna()) != len(idvotes_df[voter].dropna())) or force_update_all:

    is_ranked = votes_df[voter].iloc[0].startswith("1. ") or votes_df[voter].iloc[0].startswith("01. ")

    try: startvote = len(idvotes_df[voter].dropna()) if not update_all_votes else 0
    except KeyError: startvote = 0

    ranks, converted_votes = [], []
    for rankvote in votes_df[voter].iloc[startvote:]:
      if rankvote in ["", np.nan]: break
      if is_ranked: rank, vote = rankvote.split(". ", maxsplit=1)
      else: rank, vote = "", rankvote
      title, author = vote.split(" - ", maxsplit=1)
      author, title = unidecode.unidecode(author), unidecode.unidecode(title)
      ID, t, a = query_books_api(author, title)
      ranks.append(rank)
      converted_votes.append(ID)
      print(rank, title, author, ID, t, a)
      print("\n")

    tmp = (idvotes_df[voter].iloc[:startvote].to_list() 
    + [f"{ranks[i]}. {converted_votes[i]}" if is_ranked else f"{converted_votes[i]}" for i in range(len(ranks))])
    idvotes_df[voter] = pd.Series(tmp)

In [ ]:
display(idvotes_df.head(10))
set_with_dataframe(idvotesheet, idvotes_df, include_index=False)